In [1]:
import os
import json
from utils import json_load, json_dump, mkdir

### 1. load exam data first

In [2]:
K = 10
SOURCE_DIR = os.path.join('data', 'source')
exam_dataset_file_path = os.path.join(SOURCE_DIR, 'exam_dataset.json')
exam_data = json_load(exam_dataset_file_path)
exam_data = exam_data['examples'][:K]
exam_data[:2]

load data from: data/source/exam_dataset.json


[{'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_contexts': None,
  'reference_answer': 'B',
  'reference_answer_by': {'model_name': 're', 'type': 'ai'}},
 {'query': '題目: 依《靈樞.經脈》記載,「其直者,從巔入絡腦,還出別下項,循肩膊內,挾脊抵腰中」,指下列 何經的循行內容?\n選項:\n A: 膀胱經\n B: 膽經\n C: 胃經\n D: 肝經\n',
  'query_by': {'model_name': 're', 'type': 'ai'},
  'reference_contexts': None,
  'reference_answer': 'A',
  'reference_answer_by': {'model_name': 're', 'type': 'ai'}}]

### 2. prompt llm to extract keyword

In [3]:
import os
from dotenv import find_dotenv, load_dotenv
_ = load_dotenv(find_dotenv())

from llama_index.llms.openai import OpenAI
from llama_index.llms.ollama import Ollama

llm = OpenAI(model="gpt-5-mini", temperature=0, is_streaming=False, response_format={"type": "json_object"})
#llm = Ollama(model="gemma3:12b", temperature=0.0, request_timeout=120.0, context_window=8000, json_mode=True)

In [4]:
DEST_DIR = os.path.join('data', 'temp')
mkdir(DEST_DIR)

Directory 'data/temp' already exists.


In [5]:
save_file_path = os.path.join(DEST_DIR, 'keyword_extract_test.json')

In [6]:
from llama_index.core.prompts import PromptTemplate

QUERY_TRANSFORM_PROMPT = PromptTemplate(
    template="""
你是一個專門處理中醫考題的語言模型。我將給你一道中醫考題，請從題目和選項中提取最多 5 個「專有名詞」，
這些名詞應該是適合拿去維基百科搜尋的關鍵字，也就是在維基百科上可能有條目的中醫專有名詞，例如穴位名稱、方劑名稱、病證名稱等。  

請以 JSON 格式返回，key 為 "keyword"，value 為一個字串列表。  
不要輸出其他文字或解釋。  

範例輸出：
{
  "keyword": ["四關穴", "合谷", "太衝"]
}

#{query}

請直接輸出 JSON：
"""
)

In [7]:
rvs = []
for idx, exam in enumerate(exam_data):
    print(f'idx: {idx}')
#    if idx == 3:
#        break
    rv = {}
    query = exam['query']
    prompt = QUERY_TRANSFORM_PROMPT.format(query=query)
    # print(prompt)
    response = llm.complete(prompt)
    rv['query'] = query
    rv['reference_answer'] = exam['reference_answer']
    rv['response'] = json.loads(response.text)
    rvs.append(rv)

idx: 0
idx: 1
idx: 2
idx: 3
idx: 4
idx: 5
idx: 6
idx: 7
idx: 8
idx: 9


In [8]:
json_dump(save_file_path, rvs)

dump result to: data/temp/keyword_extract_test.json


### 3. wiki search

In [9]:
import wikipedia
class WikiSearcher:
    def __init__(self, language="zh"):
        wikipedia.set_lang(language)  # 設定語言，例如中文
        self.cache = {}

    def search_keyword(self, keyword):
        """
        查詢單個關鍵字
        返回字典: {title, url, summary}，找不到返回 None
        """
        if keyword in self.cache:
            return self.cache[keyword]

        try:
            page = wikipedia.page(keyword, auto_suggest=False)
            data = {
                "title": page.title,
                "url": page.url,
                "summary": page.summary,
                "content": page.content,
            }
            # 快取結果
            self.cache[keyword] = data
            self.cache[page.title] = data
            return data
        except wikipedia.DisambiguationError as e:
            # 多義詞，返回可能選項列表
            self.cache[keyword] = {"title": None, "url": None, "summary": None, "options": e.options}
            return self.cache[keyword]
        except wikipedia.PageError:
            # 找不到頁面
            self.cache[keyword] = None
            return None
        except Exception as e:
            # 其他錯誤
            self.cache[keyword] = None
            return None

    def search_keywords(self, keywords):
        """
        批量查詢關鍵字
        返回字典 {keyword: data}
        """
        results = {}
        for kw in keywords:
            results[kw] = self.search_keyword(kw)
        return results

In [10]:
save_file_path = os.path.join(DEST_DIR, 'wiki_search_test.json')

In [11]:
keyword_results = rvs.copy()

In [12]:
wiki_searcher = WikiSearcher(language="zh")

In [13]:
rvs = []
for idx, keyword_result in enumerate(keyword_results):
    print(f"idx: {idx}", end=', ')
    rv = keyword_result.copy()
    keywords = rv['response']['keyword']
    search_results = wiki_searcher.search_keywords(keywords)
    rv['wiki_results'] = search_results
    rvs.append(rv)

json_dump(save_file_path, rvs)

idx: 0, idx: 1, idx: 2, idx: 3, idx: 4, idx: 5, 

/home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/poyuan/miniconda3/envs/rag30/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


idx: 6, idx: 7, idx: 8, idx: 9, dump result to: data/temp/wiki_search_test.json


In [21]:
def get_context(wiki_results):
    context = ''
    for keyword, value in wiki_results.items():
        if not value:
            continue
        try:
            title = value['title']
            content = value['content']
            context+=f'-----'
            context+=f"搜索關鍵字: {keyword}\n"
            context+=f"條目名稱: {title}\n"
            context+=f"內容: {content}\n"
            context+=f'-----'
        except:
            continue
    if len(context) == 0:
        context+='沒有找到相關結果'
    return context

# 4. respnse with wiki search

In [15]:
wiki_results = rvs.copy()

In [16]:
ANSWER_PROMPT_WITH_CONTEXT = PromptTemplate(
    template="""
你是一個中醫考題專家，請根據下面的題目回答單選題。

請遵守以下規則：
1. 嚴格依據提供的參考資料 context 作答。
2. 輸出 JSON 格式。
3. JSON 需包含兩個 key：
   - "ans" ：只回答單選答案 (A/B/C/D)
   - "feedback" ：簡短說明為什麼選這個答案
4. 不要加入題目之外的說明或其他文字。

題目：
#{query}

參考資料 (context)：
#{context}

請直接輸出 JSON：
"""
)


In [25]:
llm = Ollama(model="gemma3:12b", temperature=0.0, request_timeout=120.0, context_window=8000, json_mode=True)
DEST_DIR = os.path.join('data', 'deliverables')
mkdir(DEST_DIR)
save_file_path = os.path.join(DEST_DIR, 'gemma_response_with_wiki.json')

Directory 'data/deliverables' already exists.


In [26]:
rvs = []
for idx, wiki_result in enumerate(wiki_results):
    print(f"idx: {idx}", end=', ')
    rv = wiki_result.copy()
    wiki_response = rv['wiki_results']
    rv['keywords'] = rv['response']['keyword']
    context = get_context(wiki_response)
    query = wiki_result['query']
    prompt = ANSWER_PROMPT_WITH_CONTEXT.format(query=query, context=context)
    #print(prompt)
    response = llm.complete(prompt)
    rv['context'] = context
    rv['response'] = json.loads(response.text)
    rvs.append(rv)

#json_dump(save_file_path, rvs)

idx: 0, idx: 1, idx: 2, idx: 3, idx: 4, idx: 5, idx: 6, idx: 7, idx: 8, idx: 9, 

In [27]:
json_dump(save_file_path, rvs)

dump result to: data/deliverables/gemma_response_with_wiki.json


In [28]:
rvs

[{'query': '題目: 常見針灸配穴法中,所指的「四關穴」,為下列何穴位之組合?\n選項:\n A: 上星、日月\n B: 合谷、太衝\n C: 內關、外關\n D: 上關、下關\n',
  'reference_answer': 'B',
  'response': {'ans': 'C',
   'feedback': '題目問的是「四關穴」，而參考資料中沒有提及四關穴的定義或組成。因此，無法根據提供的資料判斷正確答案。'},
  'wiki_results': {'四關穴': None,
   '合谷': {'title': '合谷穴',
    'url': 'https://zh.wikipedia.org/wiki/%E5%90%88%E8%B0%B7%E7%A9%B4',
    'summary': '合谷穴（LI 4）是手陽明大腸經的原穴，出自《靈樞·本輸》，又名虎口。“合”意即合攏，“谷”是山谷的意思。此穴在第一、二掌骨之間，兩骨相合，形狀如山谷的地方，所以名為合谷。又因位於手拇指虎口兩骨之間，所以又稱為虎口。',
    'content': '合谷穴（LI 4）是手陽明大腸經的原穴，出自《靈樞·本輸》，又名虎口。“合”意即合攏，“谷”是山谷的意思。此穴在第一、二掌骨之間，兩骨相合，形狀如山谷的地方，所以名為合谷。又因位於手拇指虎口兩骨之間，所以又稱為虎口。\n\n\n== 定位 ==\n位於手背，第一、二掌骨之間，約當第二掌骨橈側的中點處取穴。\n\n\n== 取穴法 ==\n簡便取穴法：以一手的拇指指骨關節橫紋，放在另一手拇、食指之間的指蹼緣上，當拇指尖下是穴。\n\n《針灸甲乙經》：在手大指次指歧骨間。\n《針灸大成》：在手大指次指歧骨間陷中。\n\n\n== 局部解剖 ==\n在第一、二掌骨之間，第一骨間背側肌中，深層有拇內收肌橫頭。有手背靜脈網。腧穴近側正當橈動脈從手背穿向手掌之處。布有橈神經淺支的掌背側神經，深部有正中神經的指掌側固有神經。\n\n\n== 刺灸法 ==\n直刺0.5-0.8寸，可灸。\n\n《針灸大成》：針三分, 留六呼, 灸三壯。合谷，婦人妊娠可瀉不可補，補即墮胎。\n\n\n== 主治 ==\n功能：清熱解表，明目聰耳，鎮靜止痛\n主治：\n\n本經脈所過部位的疾患：指孿，臂痛，半身不遂。\n外感疾患：

# check the answer

In [29]:
correct = 0
incorrect = 0
for rv in rvs:
    reference_answer = rv['reference_answer']
    pred_ans = rv['response']['ans']
    if reference_answer == pred_ans:
        correct+=1
    else:
        incorrect+=1

print(f"{correct}/{correct + incorrect}")

6/10
